In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os

from models.keras_ssd7 import build_model
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

In [ ]:
# Set the image size.
img_height = 200
img_width = 200

In [ ]:
img_height = 200 # Height of the input images
img_width = 200 # Width of the input images
img_channels = 3 # Number of color channels of the input images
intensity_mean = 127.5 # Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
intensity_range = 127.5 # Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
n_classes = 1 # Number of positive classes
scales = [0.08, 0.16, 0.32, 0.64, 0.96] # An explicit list of anchor box scaling factors. If this is passed, it will override `min_scale` and `max_scale`.
aspect_ratios = [0.5, 1.0, 2.0] # The list of aspect ratios for the anchor boxes
two_boxes_for_ar1 = True # Whether or not you want to generate two anchor boxes for aspect ratio 1
steps = None # In case you'd like to set the step sizes for the anchor box grids manually; not recommended
offsets = None # In case you'd like to set the offsets for the anchor box grids manually; not recommended
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [1.0, 1.0, 1.0, 1.0] # The list of variances by which the encoded target coordinates are scaled
normalize_coords = True # Whether or not the model is supposed to use coordinates relative to the image size

In [ ]:
# 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = build_model(image_size=(img_height, img_width, img_channels),
                    n_classes=n_classes,
                    mode='inference',
                    l2_regularization=0.0005,
                    scales=scales,
                    aspect_ratios_global=aspect_ratios,
                    aspect_ratios_per_layer=None,
                    two_boxes_for_ar1=two_boxes_for_ar1,
                    steps=steps,
                    offsets=offsets,
                    clip_boxes=clip_boxes,
                    variances=variances,
                    normalize_coords=normalize_coords,
                    subtract_mean=intensity_mean,
                    divide_by_stddev=intensity_range)

# 2: Load the trained weights into the model.

# TODO: Set the path of the trained weights.
# weights_path = 'ssd7_model_checkpoint.h5'
weights_path = '../XLA_1.12/ssd7_model_checkpoint.h5'

model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [ ]:
# TODO: Set the path to the `.h5` file of the model to be loaded.
model_path = '../ssd7_model_valloss_1.7'

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

K.clear_session() # Clear previous models from memory.

model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'L2Normalization': L2Normalization,
                                               'DecodeDetections': DecodeDetections,
                                               'compute_loss': ssd_loss.compute_loss})

In [ ]:
orig_images = [] # Store the images here.
input_images = [] # Store resized versions of the images here.

# We'll only load one image in this example.
for img in os.listdir("full_image/"):
#     img_path = 'full_image/IMG_3029.JPG'
    img_path = os.path.join("full_image", img)
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    print(img_path)
    plt.figure(figsize=(20,12))
    plt.imshow(img)
    plt.show()
    orig_images.append(img)
    img = cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_AREA)
    img = image.img_to_array(img) 
    input_images.append(img)
input_images = np.array(input_images)

In [ ]:
y_pred = model.predict(input_images)

In [ ]:
confidence_threshold = 0.2

y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
np.set_printoptions(precision=2, suppress=True, linewidth=90)
for i in range(len(y_pred)):
    print("Predicted boxes %d:\n"%i)
    print('   class   conf xmin   ymin   xmax   ymax')
    print(y_pred_thresh[i])

In [ ]:
for i in range(len(y_pred)):
    plt.imshow(orig_images[i])
    plt.show()
    try:
        print(i)
        box = y_pred_thresh[i][0]
        xmin = int(round(box[2] * orig_images[i].shape[1] / img_width))
        ymin = int(round(box[3] * orig_images[i].shape[0] / img_height))
        xmax = int(round(box[4] * orig_images[i].shape[1] / img_width))
        ymax = int(round(box[5] * orig_images[i].shape[0] / img_height))
        roi = orig_images[i][ymin:ymax, xmin:xmax]
        plt.imshow(roi)
        plt.show()
    except:
        continue

In [ ]:
plt.imshow(orig_images[i])
plt.show()
try:
    print(i)
    box = y_pred_thresh[i][2]
    xmin = int(round(box[2] * orig_images[i].shape[1] / img_width))
    ymin = int(round(box[3] * orig_images[i].shape[0] / img_height))
    xmax = int(round(box[4] * orig_images[i].shape[1] / img_width))
    ymax = int(round(box[5] * orig_images[i].shape[0] / img_height))
    roi = orig_images[i][ymin:ymax, xmin:xmax]
    plt.imshow(roi)
    plt.show()
except:
    print("Error")

In [ ]:
model.save("../ssd7_model_valloss_1.7")

In [ ]:
# Display the image and draw the predicted boxes onto it.

# Set the colors for the bounding boxes
colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
classes = ['background',
           'meter']

for i in range(len(y_pred)):
    plt.figure(figsize=(20,12))

    current_axis = plt.gca()
    plt.imshow(orig_images[i])
    plt.show()
    for box in y_pred_thresh[i]:
        # Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
        xmin = box[2] * orig_images[i].shape[1] / img_width
        ymin = box[3] * orig_images[i].shape[0] / img_height
        xmax = box[4] * orig_images[i].shape[1] / img_width
        ymax = box[5] * orig_images[i].shape[0] / img_height
        color = colors[int(box[0])]
        label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
        current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
        current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})